In [1]:
import pandas

file_name = 'debian_data.csv'

data = pandas.read_csv(file_name)

print(data.columns)

Index(['code', 'text', 'hash', 'label', 'filename', 'project'], dtype='object')


In [2]:
len(data['hash'])

39316

In [3]:
data[:1]

code  \
0  /*\n * PPD cache implementation for CUPS.\n *\...   

                                                text                 hash  \
0  Insufficient restriction of IPP filters in CUP...  8208116086094222111   

   label     filename project  
0  False  ppd-cache.c  debian

In [4]:
data_json = data.to_json(orient='records')

In [5]:
import json

data_array = json.loads(data_json)

In [6]:
examples = {}

for i, example in enumerate(data_array):
    
    file_key = example['project'] + ' ' + example['filename']
    if file_key not in examples:
        examples[file_key] = {}
    label = example['label']
    if label not in examples[file_key]:
        examples[file_key][label] = []
    examples[file_key][label].append(example)
    if i % 5000 == 4999:
        print(file_key)
    

chrome connection_info_popup_android.cc
chrome dtitv_impl.h
chrome CanvasAsyncBlobCreator.h
chrome node_channel.cc
chrome child_process_security_policy_impl.cc
chrome pngconf.h
chrome gles2_cmd_decoder_unittest_base.cc


In [7]:
diff_usable = []
correct_counter = 0
for key in examples.keys():
    if len(examples[key][True]) != 1 or len(examples[key][False]) != 1:
        pass
    else:
        examples[key][True] = examples[key][True][0]
        examples[key][False] = examples[key][False][0]
        correct_counter += 1
        diff_usable.append(examples[key])
print(correct_counter)
print(len(diff_usable))


4924
4924


In [8]:
#                                                          -8228664527580018723                      

json_file = open('debian_chrome_data_full.json', 'w')
json_file.write(data_json + '\n')
json_file.close()

json_file = open('debian_chrome_data_one-to-one-map.json', 'w')
json_file.write(json.dumps(diff_usable))
json_file.close()


In [9]:
import re

pattern1 = re.compile("^[0-9]+ ")
pattern2 = re.compile("^[0-9]+$")

strings = ["   this is 123 good", '123 hello world 123', '   56 /* dsjkfk*/  ', '  47 ']
for s in strings:
    m1 = pattern1.match(s.strip())
    m2 = pattern2.match(s.strip())
    if m1 is not None:
        index = s.find(m1.group())
        l = len(m1.group())
        ms = s[(index + l):]
        print(ms, len(ms))
    elif m2 is not None:
        index = s.find(m2.group())
        l = len(m2.group())
        ms = s[(index + l):]
        print(ms, len(ms))

        
        

hello world 123 15
/* dsjkfk*/   13
  1


In [10]:
def process_code(code):
    code_lines = code.split('\n')
    modified_code_lines = []
    for line in code_lines:
        trimmed_line = line.strip()
        m1 = pattern1.match(trimmed_line)
        m2 = pattern2.match(trimmed_line)
        if m1 is None and m2 is None:
            modified_code_lines.append(line)
        elif m1 is not None:
            matched_string = m1.group()
            index = line.find(matched_string)
            l = len(matched_string)
            trimmed_line = line[(index + l):]
            modified_code_lines.append(trimmed_line)
        elif m2 is not None:
            matched_string = m2.group()
            index = line.find(matched_string)
            l = len(matched_string)
            trimmed_line = line[(index + l):]
            modified_code_lines.append(trimmed_line)
    return '\n'.join(modified_code_lines)

In [11]:
import os
import sys
import difflib

if not os.path.exists('seperate_files'):
    os.mkdir('seperate_files')

if not os.path.exists('seperate_files/previous'):
    os.mkdir('seperate_files/previous')
    
if not os.path.exists('seperate_files/next'):
    os.mkdir('seperate_files/next')

count = 0
    
for i, data in enumerate(diff_usable):
    print(i)
    previous_code = data[True]['code']
    next_code = data[False]['code']
    if previous_code is None or next_code is None:
        continue
    if len(previous_code.strip()) == 0 or len(next_code.strip()) == 0:
        continue
    with open(os.path.join('seperate_files/previous', str(i + 1) + '.c'), 'w') as fp:
        p = process_code(previous_code)
#         p = previous_code
        fp.write(p)
        fp.close()
        
    with open(os.path.join('seperate_files/next', str(i + 1) + '.c'), 'w') as fp:
        p = process_code(next_code)
#         p = next_code
        fp.write(p)
        fp.close()
    count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162


3959
3960
3961
3962
3963
3964
3965
3966
3967
3968
3969
3970
3971
3972
3973
3974
3975
3976
3977
3978
3979
3980
3981
3982
3983
3984
3985
3986
3987
3988
3989
3990
3991
3992
3993
3994
3995
3996
3997
3998
3999
4000
4001
4002
4003
4004
4005
4006
4007
4008
4009
4010
4011
4012
4013
4014
4015
4016
4017
4018
4019
4020
4021
4022
4023
4024
4025
4026
4027
4028
4029
4030
4031
4032
4033
4034
4035
4036
4037
4038
4039
4040
4041
4042
4043
4044
4045
4046
4047
4048
4049
4050
4051
4052
4053
4054
4055
4056
4057
4058
4059
4060
4061
4062
4063
4064
4065
4066
4067
4068
4069
4070
4071
4072
4073
4074
4075
4076
4077
4078
4079
4080
4081
4082
4083
4084
4085
4086
4087
4088
4089
4090
4091
4092
4093
4094
4095
4096
4097
4098
4099
4100
4101
4102
4103
4104
4105
4106
4107
4108
4109
4110
4111
4112
4113
4114
4115
4116
4117
4118
4119
4120
4121
4122
4123
4124
4125
4126
4127
4128
4129
4130
4131
4132
4133
4134
4135
4136
4137
4138
4139
4140
4141
4142
4143
4144
4145
4146
4147
4148
4149
4150
4151
4152
4153
4154
4155
4156
4157
4158


In [12]:
print(next_code)


// Copyright 2016 The Chromium Authors. All rights reserved.
// Use of this source code is governed by a BSD-style license that can be
// found in the LICENSE file.

#ifndef MockFetchContext_h
#define MockFetchContext_h

#include "platform/loader/fetch/FetchContext.h"
#include "platform/loader/fetch/FetchParameters.h"
#include "platform/loader/fetch/ResourceTimingInfo.h"
#include "platform/scheduler/test/fake_web_task_runner.h"
#include "platform/wtf/PtrUtil.h"
#include "public/platform/Platform.h"

#include <memory>

namespace blink {

class KURL;
class ResourceRequest;
class WebTaskRunner;
struct ResourceLoaderOptions;

// Mocked FetchContext for testing.
class MockFetchContext : public FetchContext {
 public:
 enum LoadPolicy {
    kShouldLoadNewResource,
    kShouldNotLoadNewResource,
 };
 // TODO(toyoshim): Disallow to pass nullptr for |taskRunner|, and force to use
 // FetchTestingPlatformSupport's WebTaskRunner. Probably, MockFetchContext
 // would be available only through the 

In [13]:
import difflib
import clang.cindex
import clang.enumerations
import csv

# set the config
clang.cindex.Config.set_library_path("/usr/lib/x86_64-linux-gnu")
clang.cindex.Config.set_library_file('/usr/lib/x86_64-linux-gnu/libclang-6.0.so.1')

previous_code = '1\n2\n3\n4'
next_code = '4\n3\n2\n1'

In [14]:
pc = previous_code.split('\n')
nc = next_code.split('\n')
print(len(pc), len(nc))

4 4


In [15]:
difflib.context_diff(pc, nc)

<generator object context_diff at 0x7f1e59a967c8>

In [16]:
import re
nlp = re.compile("-\d+,")
plp = re.compile("\+\d+,")
line = "@@ -48,6 +41,14 @@"
nl = nlp.search(line).group()
pl = plp.search(line).group()


# print(int(nl[1:-1]), int(pl[1:-1]))
def parse_line_numbers(line):
    assert line.find("@@") != -1
    nl = nlp.search(line).group()
    pl = plp.search(line).group()
    return int(pl[1:-1]), int(nl[1:-1])

print(parse_line_numbers(line))
    
    
    
for x in [s for s in difflib.unified_diff(pc, nc)]:
    print('d\t' , x)

(41, 48)
d	 --- 

d	 +++ 

d	 @@ -1,4 +1,4 @@

d	 +4
d	 +3
d	 +2
d	  1
d	 -2
d	 -3
d	 -4


In [17]:
def get_added_and_deleted_line_numbers(pc, nc):
    if isinstance(pc, str):
        pc = pc.split('\n')
    if isinstance(nc, str):
        nc = nc.split('\n')
    diffs = [s for s in difflib.unified_diff(pc, nc)]
    if len(diffs) == 0:
        return [], []
    added = []
    deleted = []
    diffs = diffs[2:]
    cal = 0
    cdl = 0
    for i in range(len(diffs)):
        dl = diffs[i]
        if dl.startswith('@@'):
            cal, cdl = parse_line_numbers(dl)
#             print(cal, cdl, '\t found for\t', dl.strip())
#             print('-------------------------------------------')
        elif not dl.startswith('+') and not dl.startswith('-'):
#             print('Found\t', dl)
#             print('Current lns\t', cal, cdl)
#             print('Moving on')
#             print('============================================')
            cal += 1
            cdl += 1
        elif dl.startswith('+'):
#             print('Found Adding Line \t', dl)
#             print('Current lns\t', cal)
#             print('Adding to the addtions')
#             print('Moving On')
#             print('********************************************')
            added.append(cal)
            cal += 1
        elif dl.startswith('-'):
#             print('Found Deletions Line \t', dl)
#             print('Current lns\t', cdl)
#             print('Adding to the Deletions')
#             print('Moving On')
#             print('#############################################')
            deleted.append(cdl)
            cdl += 1
    return added, deleted

a, b = get_added_and_deleted_line_numbers(pc, nc)
print(a)
print(b)
        

[1, 2, 3]
[2, 3, 4]


In [18]:
def read_file(path):
    file_text = ''
    with open(path) as fp:
        for line in fp:
            file_text += line
    return file_text

nc = read_file("seperate_files/next/221.c")
pc = read_file("seperate_files/previous/221.c")
a, b = get_added_and_deleted_line_numbers(pc, nc)
print(a)
print(b)

[31, 233, 240, 241, 325, 326, 362]
[232, 233, 234, 235, 242, 326, 355, 356, 364, 365]


In [19]:
class Tokenizer:
    # creates the object, does the inital parse
    def __init__(self, path, tokenizer_type='original'):
        self.index = clang.cindex.Index.create()
        self.tu = self.index.parse(path)
        self.path = self.extract_path(path)
        self.symbol_table = {}
        self.symbol_count = 1
        self.tokenizer_type = tokenizer_type
    
    # To output for split_functions, must have same path up to last two folders
    def extract_path(self, path):
        return "".join(path.split("/")[:-2])

    # does futher processing on a literal token  
    def process_literal(self, literal):
        cursor_kind = clang.cindex.CursorKind
        kind = literal.cursor.kind
        #print literal.spelling
    
        if kind == cursor_kind.INTEGER_LITERAL:
            return literal.spelling
            #return ["NUM"]

        if kind == cursor_kind.FLOATING_LITERAL:
            return ["NUM"]

        if kind == cursor_kind.IMAGINARY_LITERAL:
            return ["NUM"]

        if kind == cursor_kind.STRING_LITERAL:
            return ["STRING"]

        if kind == cursor_kind.CHARACTER_LITERAL:
            return ["CHAR"]

        if kind == cursor_kind.CXX_BOOL_LITERAL_EXPR:
            return ["BOOL"]

        # catch all other literals
        return ["LITERAL"]
    
    # filters out unwanted punctuation    
    def process_puntuation(self, punctuation):
        spelling = punctuation.spelling          
        return [spelling]
    
    # further processes and identifier token    
    def process_ident(self, ident):
        # assert type in ['original', 'symbolic', 'flat'], 'Token type must be original, symbolic, flat'
        # are we a "special" ident?
        # if ident.spelling in ["std", "cout", "cin", "vector", "pair", "string", "NULL", "size_t"]:
        return [ident.spelling]
#         ident_name = ident.spelling
#         if self.tokenizer_type == 'original':
#             return [ident_name]
#         elif self.tokenizer_type == 'symbolic':
#             if ident_name not in self.symbol_table:
#                 symbolic_name = 'ID' + str(self.symbol_count)
#                 self.symbol_table[ident_name] = symbolic_name
#                 self.symbol_count += 1
#             return [self.symbol_table[ident_name]]
#         # catch all others
#         else:
#             return ["ID"]
    
    # tokenizes the contents of a specific cursor
    def full_tokenize_cursor(self, cursor):
        tokens = cursor.get_tokens()
        # return final tokens as a list
        result = []

        for token in tokens:
            if token.kind.name == "COMMENT":
                continue
    
            else:
                spelling = token.spelling
                spelling = re.sub('\s+', ' ', spelling)
                
#                 if spelling in [';', '{', '}']:
#                     spelling += '\n'
                result += [spelling]
                continue
    
#             if token.kind.name == "IDENTIFIER":
#                 result += self.process_ident(token)
#                 continue
    
#             if token.kind.name == "KEYWORD":
#                 result += [token.spelling]
    
        return result
    
    def extract_line_numbers(self, cursor):
        tokens = cursor.get_tokens()
        lines = []
        for token in tokens:
            lines.append(token.location.line)
        return list(sorted(set(lines)))
    
    # tokenizes the entire document
    def full_tokenize(self):
        cursor = self.tu.cursor
        return self.full_tokenize_cursor(cursor)
    
    def extract_function_prototype(self, c):
        cursor_kind = clang.cindex.CursorKind        
        assert c.kind == cursor_kind.CXX_METHOD or c.kind == cursor_kind.FUNCTION_DECL
        name = c.spelling
        rtype = c.result_type.spelling
        args = []
        for arg in c.get_arguments():
            args.append(str(arg.type.spelling) + " " + str(arg.spelling))
        return str(rtype) + " " + str(name) + " (" + ', '.join(args) + ")"
    
    # returns a list of function name / function / filename tuples
    
    def extract_method_and_line_numbers(self):
        results = {}
        cursor_kind = clang.cindex.CursorKind
        
        cursor = self.tu.cursor
        for c in cursor.get_children():
            if c.kind == cursor_kind.CXX_METHOD or c.kind == cursor_kind.FUNCTION_DECL:
                name = self.extract_function_prototype(c)
                lines = self.extract_line_numbers(c)
                tokens = ' '.join(self.full_tokenize_cursor(c))
                tokens = re.sub('\s+', ' ', tokens)
                tokens = re.sub('{', '{\n', tokens)
                tokens = re.sub(';', ';\n', tokens)
                tokens = re.sub('}', '}\n', tokens)
                results[name] = (lines, tokens)
                
        return results
    
    def split_functions(self, method_only):
        results = []
        cursor_kind = clang.cindex.CursorKind
        
        # query all children for methods, and then tokenize each
        cursor = self.tu.cursor
        for c in cursor.get_children():
            filename = c.location.file.name if c.location.file != None else "NONE"
            extracted_path = self.extract_path(filename)
            
            if (c.kind == cursor_kind.CXX_METHOD or (method_only == False and c.kind == cursor_kind.FUNCTION_DECL)) and extracted_path == self.path:
                name = c.spelling
#                 tokens = self.full_tokenize_cursor(c)
#                 filename = filename.split("/")[-1]
#                 results += [tokens]
                lines = self.extract_line_numbers(c)

                print(self.extract_function_prototype(c), '\t->\n', lines)
                
        return results
    
f1 = "/home/saikatc/DATA/vulnerable-code/code-slicer/test/test1.c"
f2 = "seperate_files/next/221.c"
tok = Tokenizer(f2)
print(tok.extract_method_and_line_numbers())

{'void tscc2_idct4_put (int * in, int [3] q, int * dst, int stride)': ([101, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122], 'static void tscc2_idct4_put ( int * in , int q [ 3 ] , uint8_t * dst , int stride ) {\n int i ;\n int tblk [ 4 * 4 ] ;\n int t0 , t1 , t2 , t3 ;\n for ( i = 0 ;\n i < 4 ;\n i ++ ) {\n t0 = DEQUANT ( q [ 0 + ( i & 1 ) ] , in [ 0 * 4 + i ] ) ;\n t1 = DEQUANT ( q [ 1 + ( i & 1 ) ] , in [ 1 * 4 + i ] ) ;\n t2 = DEQUANT ( q [ 0 + ( i & 1 ) ] , in [ 2 * 4 + i ] ) ;\n t3 = DEQUANT ( q [ 1 + ( i & 1 ) ] , in [ 3 * 4 + i ] ) ;\n DCT1D ( tblk [ 0 * 4 + i ] , tblk [ 1 * 4 + i ] , tblk [ 2 * 4 + i ] , tblk [ 3 * 4 + i ] , t0 , t1 , t2 , t3 , COL_OP ) ;\n }\n for ( i = 0 ;\n i < 4 ;\n i ++ ) {\n DCT1D ( dst [ 0 ] , dst [ 1 ] , dst [ 2 ] , dst [ 3 ] , tblk [ i * 4 + 0 ] , tblk [ i * 4 + 1 ] , tblk [ i * 4 + 2 ] , tblk [ i * 4 + 3 ] , ROW_OP ) ;\n dst += stride ;\n }\n }\n'), 'int tscc2_decode_mb (TSCC2Context * c, int * q, 

In [20]:
# for i, data in enumerate(diff_usable):
#     if i % 3 == 2:
#         break
#     previous_code = data[True]['code']
#     next_code = data[False]['code']
#     if previous_code is None or next_code is None:
#         continue
#     if len(previous_code.strip()) == 0 or len(next_code.strip()) == 0:
#         continue
        
#     pc = process_code(previous_code)
#     nc = process_code(next_code)
#     pfp = os.path.join('seperate_files/previous', str(i + 1) + '.c')
#     nfp = os.path.join('seperate_files/next', str(i + 1) + '.c')
# #     with open(pfp, 'w') as fp:
# #         fp.write(pc)
# #         fp.close()
        
# #     with open(nfp, 'w') as fp:
# #         fp.write(nc)
# #         fp.close()
#     count += 1


def compressed_text(text):
    return text.replace(' ', '').replace('\n', '').replace('\t', '')

def extract_selected_lines_from_file_text_string(fp_text, lines):
    fp_lines = fp_text.split('\n')
    return_text = ''
    for i in range(len(fp_lines)):
        if (i + 1) in lines:
            return_text += (fp_lines[i] + '\n')
    return return_text, compressed_text(return_text)

# print(extract_selected_lines_from_file_text_string(
#     "chrome connection_info_popup_android.cc\nchrome dtitv_impl.h\n" + \
#     "chrome CanvasAsyncBlobCreator.h\nchrome node_channel.cc" + \
#     "\nchrome child_process_security_policy_impl.cc\nchrome pngconf.h\nchrome gles2_cmd_decoder_unittest_base.cc",
#     [2,4]
# ))


In [21]:
import json

json_file = open('debian_chrome_data_one-to-one-map.json')
diff_usables = json.load(json_file)
json_file.close()



In [23]:
import os, sys

vulnerable_functions = []
non_vulnerable_functions = []
vulnerable_num_lines = []
non_vulnerable_num_lines = []

vulnerable_set = set()
non_vulnerable_set = set()

for i in range(len(diff_usables)):
    if i % 25 == 24:
        print(i, len(vulnerable_functions), len(non_vulnerable_functions))
    pfp = os.path.join('seperate_files/previous', str(i + 1) + '.c')
    nfp = os.path.join('seperate_files/next', str(i + 1) + '.c')
    if not os.path.exists(pfp) or not os.path.exists(nfp):
        continue
    pc = read_file(pfp)
    nc = read_file(nfp)
    
    pdp = diff_usables[i]['true']
    ndp = diff_usables[i]['false']
    
    prev_tokenizer = Tokenizer(pfp)
    previous_version_method_lines = prev_tokenizer.extract_method_and_line_numbers()
    
    next_tokenizer = Tokenizer(nfp)
    next_version_method_lines = next_tokenizer.extract_method_and_line_numbers()
    
    previous_version_method_names = set(previous_version_method_lines.keys())
    next_version_method_names = set(next_version_method_lines.keys())
    
    common_methods = previous_version_method_names.intersection(next_version_method_names)
    only_previous = previous_version_method_names.difference(next_version_method_names)
    only_next = next_version_method_names.difference(previous_version_method_names)
    
    added_lines, deleted_lines = get_added_and_deleted_line_numbers(pc, nc)
#     for function_prototype in only_next:
#         lines, text = next_version_method_lines[function_prototype]
#         # text, ctext = extract_selected_lines_from_file_text_string(nc, lines)
#         ctext = compressed_text(text)
#         if len(text.strip()) == 0:
#                 continue
#         if ctext not in non_vulnerable_set and len(text.strip().split('\n')) > 1:
#             non_vulnerable_functions.append(
#                 {
#                     'code': text.strip(),
#                     'hash': ndp['hash'],
#                     'project': ndp['project'],
#                     'size': len(text.strip().split('\n'))
                    
#                 }
#             )
#             non_vulnerable_set.add(ctext)
#             non_vulnerable_num_lines.append(len(lines))
        
    for function_prototype in only_previous:
        lines, text = previous_version_method_lines[function_prototype]
        # text, ctext = extract_selected_lines_from_file_text_string(pc, lines)
        ctext = compressed_text(text)
        if len(text.strip()) == 0:
                continue
        if ctext not in vulnerable_set and len(text.strip().split('\n')) > 1:    
            vulnerable_functions.append(
                {
                    'code': text.strip(),
                    'hash': pdp['hash'],
                    'project': pdp['project'],
                    'size': len(text.strip().split('\n'))
                }
            )
            vulnerable_set.add(ctext)
            vulnerable_num_lines.append(len(lines))
    
    for function_prototype in common_methods:
        previous_version_lines, prev_text = previous_version_method_lines[function_prototype]
        next_version_lines, next_text = next_version_method_lines[function_prototype]
        lines_deleted_from_prev = set(previous_version_lines).intersection(set(deleted_lines))
        lines_added_to_next = set(next_version_lines).intersection(set(added_lines))
        if len(lines_added_to_next) > 0 or len(lines_deleted_from_prev) > 0:
            # text, ctext = extract_selected_lines_from_file_text_string(pc, previous_version_lines)
            text = prev_text
            ctext = compressed_text(text)
            if len(text.strip()) == 0:
                continue
            if ctext not in vulnerable_set and len(text.strip().split('\n')) > 1:
                vulnerable_functions.append(
                    {
                        'code': text.strip(),
                        'hash': pdp['hash'],
                        'project': pdp['project'],
                        'size': len(text.strip().split('\n'))

                    }
                )
                vulnerable_set.add(ctext)
                vulnerable_num_lines.append(len(previous_version_lines))
                
            # text, ctext = extract_selected_lines_from_file_text_string(nc, next_version_lines)
            text = next_text
            ctext = compressed_text(text)
            if len(text.strip()) == 0:
                continue
            if ctext not in non_vulnerable_set and len(text.strip().split('\n')) > 1:
                non_vulnerable_functions.append(
                    {
                        'code': text.strip(),
                        'hash': ndp['hash'],
                        'project': ndp['project'],
                        'size': len(text.strip().split('\n'))
                    }
                )
                non_vulnerable_set.add(ctext)
                non_vulnerable_num_lines.append(len(next_version_lines))
        else:
            # text, ctext = extract_selected_lines_from_file_text_string(pc, previous_version_lines)
            text = prev_text
            ctext = compressed_text(text)
            if len(text.strip()) == 0:
                continue
            if ctext not in non_vulnerable_set and len(text.strip().split('\n')) > 1:
                non_vulnerable_functions.append(
                    {
                        'code': text.strip(),
                        'hash': pdp['hash'],
                        'project': pdp['project'],
                        'size': len(text.strip().split('\n'))
                    }
                )
                non_vulnerable_set.add(ctext)
                non_vulnerable_num_lines.append(len(previous_version_lines))
                
print(vulnerable_functions[0])
print(non_vulnerable_functions[0])

24 141 141
49 232 232
74 262 262
99 288 288
124 315 315
149 369 369
174 403 403
199 429 429
224 476 476
249 508 508
274 556 556
299 594 594
324 628 628
349 653 653
374 676 676
399 710 710
424 744 744
449 760 760
474 811 811
499 832 832
524 872 872
549 895 895
574 926 926
599 962 963
624 976 977
649 993 994
674 1059 1060
699 1082 1083
724 1101 1102
749 1115 1116
774 1135 1136
799 1159 1160
824 1182 1183
849 1225 1225
874 1244 1244
899 1256 1256
924 1258 1258
949 1320 1320
974 1361 1361
999 1401 1401
1024 1472 1472
1049 1512 1512
1074 1512 1512
1099 1512 1512
1124 1513 1513
1149 1514 1514
1174 1520 1520
1199 1527 1527
1224 1527 1527
1249 1528 1528
1274 1528 1528
1299 1528 1528
1324 1528 1528
1349 1528 1528
1374 1530 1530
1399 1530 1530
1424 1530 1530
1449 1530 1530
1474 1566 1566
1499 1575 1575
1524 1575 1575
1549 1575 1575
1574 1575 1575
1599 1575 1575
1624 1600 1600
1649 1601 1601
1674 1605 1605
1699 1605 1605
1724 1606 1606
1749 1606 1606
1774 1607 1608
1799 1609 1610
1824 1620 1621
1

In [24]:
print(vulnerable_functions[1]['code'])
print(non_vulnerable_functions[1]['code'])
import json
vjson = open('diff_only_data/vulnerables.json', 'w')
vjson.write(json.dumps(vulnerable_functions))
vjson.close()


nvjson = open('diff_only_data/non-vulnerables.json', 'w')
nvjson.write(json.dumps(non_vulnerable_functions))
nvjson.close()


static int jbig2_word_stream_buf_get_next_word ( Jbig2WordStream * self , int offset , uint32_t * word ) {
 Jbig2WordStreamBuf * z = ( Jbig2WordStreamBuf * ) self ;
 const byte * data = z -> data ;
 uint32_t result ;
 if ( offset + 4 < z -> size ) result = ( data [ offset ] << 24 ) | ( data [ offset + 1 ] << 16 ) | ( data [ offset + 2 ] << 8 ) | data [ offset + 3 ] ;
 else if ( offset > z -> size ) return - 1 ;
 else {
 int i ;
 result = 0 ;
 for ( i = 0 ;
 i < z -> size - offset ;
 i ++ ) result |= data [ offset + i ] << ( ( 3 - i ) << 3 ) ;
 }
 * word = result ;
 return 0 ;
 }
static int jbig2_word_stream_buf_get_next_word ( Jbig2WordStream * self , size_t offset , uint32_t * word ) {
 Jbig2WordStreamBuf * z = ( Jbig2WordStreamBuf * ) self ;
 const byte * data = z -> data ;
 uint32_t result ;
 if ( offset + 4 < z -> size ) result = ( data [ offset ] << 24 ) | ( data [ offset + 1 ] << 16 ) | ( data [ offset + 2 ] << 8 ) | data [ offset + 3 ] ;
 else if ( offset > z -> size ) return - 

In [27]:
print(vulnerable_functions[1])

{'code': 'static int jbig2_word_stream_buf_get_next_word ( Jbig2WordStream * self , int offset , uint32_t * word ) {\n Jbig2WordStreamBuf * z = ( Jbig2WordStreamBuf * ) self ;\n const byte * data = z -> data ;\n uint32_t result ;\n if ( offset + 4 < z -> size ) result = ( data [ offset ] << 24 ) | ( data [ offset + 1 ] << 16 ) | ( data [ offset + 2 ] << 8 ) | data [ offset + 3 ] ;\n else if ( offset > z -> size ) return - 1 ;\n else {\n int i ;\n result = 0 ;\n for ( i = 0 ;\n i < z -> size - offset ;\n i ++ ) result |= data [ offset + i ] << ( ( 3 - i ) << 3 ) ;\n }\n * word = result ;\n return 0 ;\n }', 'hash': -1768728156572022708, 'project': 'debian', 'size': 16}


In [28]:
base = 'diff_only_data/separate_files/'
count = 0
for v in vulnerable_functions:
    code = v['code']
    project = v['project']
    file_name = base + str(count) + '_' + project + '_1.c'
    count += 1
    with open(file_name, 'w') as fp:
        fp.write(code)
        fp.close()

for v in non_vulnerable_functions:
    code = v['code']
    project = v['project']
    file_name = base + str(count) + '_' + project + '_0.c'
    count += 1
    with open(file_name, 'w') as fp:
        fp.write(code)
        fp.close()